In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

2023-01-05 19:09:30.727969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 19:09:30.813429: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-05 19:09:31.318844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-05 19:09:31.318904: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
__name__ = "RNN"

# Model

In [3]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


In [4]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [5]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [6]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [7]:
model = getModel()
model.fit(X_train, y_train, epochs=260, callbacks=[tb_callback])

2023-01-05 19:09:32.900840: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2023-01-05 19:09:32.900875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tata-Legion-5-Pro-16ITH6
2023-01-05 19:09:32.900879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tata-Legion-5-Pro-16ITH6
2023-01-05 19:09:32.901035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2023-01-05 19:09:32.901048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.65.1
2023-01-05 19:09:32.901051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 515.65.1 does not match DSO version 510.108.3 -- cannot find working devices in thi

Epoch 1/260
2/2 [==============================] - 2s 36ms/step - loss: 0.8586 - categorical_accuracy: 0.4912
Epoch 2/260
2/2 [==============================] - 0s 28ms/step - loss: 1.5594 - categorical_accuracy: 0.5263
Epoch 3/260
2/2 [==============================] - 0s 27ms/step - loss: 3.6104 - categorical_accuracy: 0.5263
Epoch 4/260
2/2 [==============================] - 0s 28ms/step - loss: 3.9024 - categorical_accuracy: 0.5088
Epoch 5/260
2/2 [==============================] - 0s 34ms/step - loss: 0.8126 - categorical_accuracy: 0.6667
Epoch 6/260
2/2 [==============================] - 0s 28ms/step - loss: 1.5139 - categorical_accuracy: 0.5263
Epoch 7/260
2/2 [==============================] - 0s 26ms/step - loss: 1.1348 - categorical_accuracy: 0.5965
Epoch 8/260
2/2 [==============================] - 0s 27ms/step - loss: 2.8856 - categorical_accuracy: 0.4561
Epoch 9/260
2/2 [==============================] - 0s 27ms/step - loss: 1.4753 - categorical_accuracy: 0.6140
Epoch 10/2

In [8]:

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2

In [9]:
model.save(f"{modelPath}/{__name__}/test.h5")